# Local API

In [2]:
import json, requests, time, pandas as pd
import sys, os
from pathlib import Path
import requests

In [3]:
# locate project root (folder that contains app/main.py and models/logreg_meta.json)
def find_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app" / "main.py").exists() and (p / "models" / "logreg_meta.json").exists():
            return p
    raise FileNotFoundError("Cannot find project root (needs app/main.py and models/logreg_meta.json).")

ROOT = find_root(Path.cwd())
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)

Project root: c:\Users\Christopher Hanryan\Jupyter\Pandas\BreastCancer


In [4]:
from app.model.model import META, FEATURES, THRESH, predict_pipeline, __version__ as MODEL_VERSION

In [5]:
# Base url for running server
url = "http://127.0.0.1:8000"

In [6]:
%%time 
print("health:", requests.get(f"{url}/").json())

health: {'health_check': 'OK', 'model_version': '2025-09-11'}
CPU times: total: 15.6 ms
Wall time: 41.6 ms


In [7]:
RECORD = META.get("example_record") 

In [8]:
r = requests.post(f"{url}/predict", json=RECORD)
print(r.status_code, r.json())

200 {'proba': 0.006093971191650225, 'prediction': 0, 'threshold': 0.5}


# Docker API

In [27]:
url = "http://localhost:8000"

In [28]:
print(requests.get(f"{url}/").json())

{'health_check': 'OK', 'model_version': '2025-09-11'}


In [ ]:
payload = META.get("example_record")
r = requests.post(f"{url}/predict", json=payload, timeout=10)
print("predict:", r.status_code, r.json())

predict: 200 {'proba': 0.006093971191650225, 'prediction': 0, 'threshold': 0.5}


# (AWS) Docker API

In [32]:
url = "http://3.107.46.160:8000"

In [33]:
for p in (8000, 80):
    try:
        print(p, requests.get(f"http://3.107.46.160:{p}/", timeout=5).status_code)
    except Exception as e:
        print(p, type(e).__name__)

8000 200
80 ConnectTimeout


In [34]:

r = requests.get(f"{url}/")
print(f"HTTP latency: {r.elapsed.total_seconds()*1000:.1f} ms")

HTTP latency: 218.0 ms


In [35]:
%%time 
print("health:", requests.get(f"{url}/").json())

health: {'health_check': 'OK', 'model_version': '2025-09-11'}
CPU times: total: 15.6 ms
Wall time: 227 ms


In [36]:
t0 = time.perf_counter()
payload = META.get("example_record")
r = requests.post(f"{url}/predict", json=payload)
dt = (time.perf_counter() - t0) * 1000
print("status:", r.status_code, "latency_ms:", f"{dt:.1f}", "body:", r.json())

status: 200 latency_ms: 240.9 body: {'proba': 0.006093971191650225, 'prediction': 0, 'threshold': 0.5}
